In [1]:
# Dependencies
import pandas as pd
import numpy as np

# File to Load
school_csv = "Resources/schools_complete.csv"
student_csv = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_df = pd.read_csv(school_csv)
student_df = pd.read_csv(student_csv)

# Combine the data into a single dataset
merged_df = pd.merge(student_df, school_df, how="left", on=["school_name", "school_name"])
merged_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Total number of schools

* Total number of students

* Total budget

* Average math score 

* Average reading score

* The overall passing rate (overall average score)

* Percentage of students with a passing math score (70 or greater)

* Percentage of students with a passing reading score (70 or greater)

* Dataframe to hold the above results

In [2]:
# total number of schools
total_schools = len(merged_df["school_name"].unique())
# total number of students
total_students = len(merged_df["Student ID"].unique())
# total budget
total_budget=sum(school_df["budget"])
# average math score
average_math_score = sum(student_df["math_score"])/total_students
# average reading score
average_reading_score = sum(student_df["reading_score"])/total_students
# overall passing rate (overall average score)
avg_both_subjects=(average_math_score+average_reading_score)/2
# percentage of students with a passing math score (70 or greater)
passing_math_score=merged_df.loc[merged_df["math_score"]>=70]
total_number_passing_math_score=len(passing_math_score["Student ID"].unique())
total_percentage_passing_math_score=(total_number_passing_math_score/total_students)*100
# percentage of students with a passing reading score (70 or greater)
passing_reading_score=merged_df.loc[merged_df["reading_score"]>=70]
total_number_passing_reading_score=len(passing_reading_score["Student ID"].unique())
total_percentage_passing_reading_score=(total_number_passing_reading_score/total_students)*100
# Create a dataframe to hold the above results
district_summary_df = pd.DataFrame({
    "Total Schools":[total_schools],
    "Total Students":[total_students],
    "Total Budget":[total_budget],
    "Average Math Score":[average_math_score],
    "Average Reading Score":[average_reading_score],
    "% Passing Math":[total_percentage_passing_math_score],
    "% Passing Reading":[total_percentage_passing_reading_score],
    "% Overall Passing Rate": [avg_both_subjects]
})
# Optional: give the displayed data cleaner formatting
district_summary_df["Total Budget"]=district_summary_df["Total Budget"].astype(float).map("${:,.2f}".format)
district_summary_df["Total Students"]=district_summary_df["Total Students"].map("{:,.0f}".format)

district_summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.431606


## School Summary

  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * Percentage of students with a passing math score (70 or greater)
  * Percentage of students with a passing reading score (70 or greater)
  * The overall passing rate (overall average score)

In [15]:
# Dataframe for each school name
school_summary = merged_df.groupby(["school_name"])
# School Name
school_names = merged_df.school_name.sort_values().unique()
# School Type
school_types = school_df.sort_values(by="school_name").type
# Total Students Per School
total_students_per_school = list(school_summary.student_name.count())
# Total School Budget
total_school_budget = list(school_summary.budget.mean())
# Per Student Budget
school_per_student_budget = [i/j for i,j in zip(total_school_budget,total_students_per_school)]
# Average Math Score
avg_math_score_school = list(school_summary.math_score.mean())
# Average Reading Score
avg_reading_score_school = list(school_summary.reading_score.mean())

# Calculating passing percentages per subject

# % Passing Math
school_summary = merged_df[merged_df["math_score"] >= 70].groupby(["school_name"])
school_percentage_passing_math = [(i/j)*100 for i,j in zip(school_summary.math_score.count(),total_students_per_school)]
# % Passing Reading
school_summary = merged_df[merged_df["reading_score"] >= 70].groupby(["school_name"])
school_percentage_passing_reading = [(i/j)*100 for i,j in zip(school_summary.reading_score.count(),total_students_per_school)]
# Overall Passing Rate (Average of the above two)
school_overall_passing = [(i+j)/2 for i,j in zip(school_percentage_passing_math,school_percentage_passing_reading)]

# Compile all calculations into dataframe
school_summary_df = pd.DataFrame({"School Names":school_names,
                                  "School Type":school_types,
                                  "Total Students":total_students_per_school,
                                  "Total School Budget":total_school_budget,
                                  "Per Student Budget":school_per_student_budget,
                                  "Average Math Score":avg_math_score_school,
                                  "Average Reading Score":avg_reading_score_school,
                                  "% Passing Math":school_percentage_passing_math,
                                  "% Passing Reading":school_percentage_passing_reading,
                                  "%Overall Passing Rate":school_overall_passing})

school_summary_df = school_summary_df.reset_index(drop=True)
school_summary_df.head()

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,%Overall Passing Rate
0,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,74.306672
1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
2,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
3,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668


## Top Performing Schools (By Passing Rate)

* Displaying the five best-performing schools

In [16]:
# Top 5 (By Passing Rate)
top_5 = school_summary_df.sort_values(by="%Overall Passing Rate", ascending=False).head(5).reset_index(drop=True)
top_5

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,%Overall Passing Rate
0,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,95.586652
1,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,95.290520
2,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,95.270270
3,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,95.265668
4,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,95.203679


## Bottom Performing Schools (By Passing Rate)

* Displaying the five worst-performing schools

In [7]:
bottom_5 = school_summary_df.sort_values(by="%Overall Passing Rate", ascending=True).head(5).reset_index(drop=True)
bottom_5

,School Names,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,%Overall Passing Rate
0,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,73.293323
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.988471,80.739234,73.363852
2,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.683922,81.316421,73.500171
3,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.057551,81.222432,73.639992
4,Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.309602,79.299014,73.804308


## Math Scores by Grade

  * Table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Pandas series for each grade.
  
  * Group each series by school
  
  * Combine the series into a dataframe

In [19]:
def average_math_grade(grade):
    school_summary = merged_df.loc[merged_df.grade == grade].groupby(["school_name"])
    school_names = merged_df.school_name.sort_values().unique()
    school_average_math_score = list(school_summary.math_score.mean())

    # Combine the series into a dataframe
    average_math_df = pd.DataFrame({"School Names":school_names,
                                    f"{grade} ":school_average_math_score})

    average_math_df = average_math_df.reset_index(drop=True)
    return average_math_df


#Create series for each grade and group by school
grade_9 = average_math_grade("9th")
grade_10 = average_math_grade("10th")
grade_11 = average_math_grade("11th")
grade_12 = average_math_grade("12th")
avg_math_score_grade = pd.merge(grade_9,grade_10,how='inner',suffixes=('',''))
avg_math_score_grade = pd.merge(avg_math_score_grade,grade_11,how='inner')
avg_math_score_grade = pd.merge(avg_math_score_grade,grade_12,how='inner')
avg_math_score_grade.head()

,School Names,9th,10th,11th,12th
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164


## Reading Score by Grade 

  * Table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Pandas series for each grade.
  
  * Group each series by school
  
  * Combine the series into a dataframe

In [20]:
def average_reading_grade(grade):
    school_summary = merged_df.loc[merged_df.grade == grade].groupby(["school_name"])
    school_names = merged_df.school_name.sort_values().unique()
    school_avg_reading_score = list(school_summary.reading_score.mean())

    # Combine the series into a dataframe
    average_reading_df = pd.DataFrame({"School Names":school_names,
                                    f"{grade} ":school_avg_reading_score})

    average_reading_df = average_reading_df.reset_index(drop=True)
    return average_reading_df

# Create series for each grade and group by school
grade_9 = average_reading_grade("9th")
grade_10 = average_reading_grade("10th")
grade_11 = average_reading_grade("11th")
grade_12 = average_reading_grade("12th")
avg_reading_score_grade = pd.merge(grade_9,grade_10,how='inner',suffixes=('',''))
avg_reading_score_grade = pd.merge(avg_reading_score_grade,grade_11,how='inner',suffixes=('',''))
avg_reading_score_grade = pd.merge(avg_reading_score_grade,grade_12,how='inner',suffixes=('',''))
avg_reading_score_grade.head()

,School Names,9th,10th,11th,12th
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699


## Scores by School Spending

* Table that breaks down school performances based on average Spending Ranges (Per Student). 4 reasonable bins to group school spending. In the table is included each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [22]:
# Sample bins
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

scores_by_spending = school_summary_df[["School Names",
                                        "Average Math Score",
                                        "Average Reading Score",
                                        "% Passing Math",
                                        "% Passing Reading",
                                        "%Overall Passing Rate"]]
scores_by_spending["Spending Ranges (Per Student)"] = pd.cut(school_summary_df["Per Student Budget"], spending_bins, labels=group_names)


scores_by_spending = scores_by_spending.groupby(["Spending Ranges (Per Student)"])
scores_by_spending.mean()

C:\Users\maria\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,%Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Table that breaks down school performances based on school size. 4 reasonable bins to group school size. In the table is included each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [23]:
# Sample bins
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [24]:
scores_by_school_size = school_summary_df[["School Names",
                                           "Average Math Score",
                                           "Average Reading Score",
                                           "% Passing Math",
                                           "% Passing Reading",
                                           "%Overall Passing Rate"]]
scores_by_school_size["School Size"] = pd.cut(school_summary_df["Total Students"], size_bins, labels=group_names)
scores_by_school_size = scores_by_school_size.groupby(["School Size"])
scores_by_school_size.mean()

C:\Users\maria\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,%Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Table that breaks down school performances based on school type. In the table is included each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [25]:
scores_type = school_summary_df[["School Names",
                                 "School Type",
                                 "Average Math Score",
                                 "Average Reading Score",
                                 "% Passing Math",
                                 "% Passing Reading",
                                 "%Overall Passing Rate"]]
scores_type = scores_type.groupby("School Type")
scores_type.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,%Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
